# Integrating python and SQL

## Useful links:

- [Python Connector Docs](https://dev.mysql.com/doc/connector-python/en/)
- [Helpful tutorial](https://realpython.com/python-mysql/)

Python can be used in conjuction with mysql using the mysql package. This can offer more control over SQL but can also allow you to form pandas DataFrames from SQL tables, and vice versa. To use mysql you will first need to install it into your python interpreter, using `pip install mysql-connector-python`. But more about pip and installing packages in another lesson.

First thing to do is import the connector module, which allows us to connect to an SQL database from python, and then set up a connection to our chosen database:

In [1]:
import mysql.connector as Connector
Conn = Connector.connect(host='localhost', user='kiara', password='kiara_password')

Next, we instantiate a 'cursor' object, which we use to execute queries to our database through our connection.

In [2]:
Cursor = Conn.cursor()

Using the cursor object is very simple: you just insert a string that holds the SQL query you what to perform. **Note:** it is not necessary to include a ';' at the end of a query here. I want to show all databases we have stored:

In [3]:
Cursor.execute('SHOW DATABASES')
Tables = Cursor.fetchall()
Tables

[('information_schema',), ('kiara_db',)]

So we see that there are two databases avialable to us as the user 'kiara'. The `Cursor.fetchall()` method fetches the result of our query into python objects which can then be manipulated. However we could create a buffered cursor object which automatically fetches the result, but leaves it as an iterable:

In [4]:
Buff_Cursor = Conn.cursor(buffered = True)
Buff_Cursor.execute('SHOW DATABASES')

for database in Buff_Cursor:
    print(database)
    
Buff_Cursor.close();

('information_schema',)
('kiara_db',)


You can choose to do this to save some time and lines of code, and if you aren't bothered about saving the result directly into a python variable.

When I established the connection in cell 1, I did not specify which database we want to use and so we have not got one automatically selected. Just like in an SQL terminal. However we can connect instantly to a specific database. First we have to close our current cursor (this resets the cursor essentially) and cut off our current connection to SQL:

In [5]:
Cursor.close()
Conn.close()

We can now reconnect specifically to a database called 'kiara_db', using the `database` keyword:

In [6]:
Conn = Connector.connect(host='localhost', user='kiara', password='kiara_password', database='kiara_db')
Cursor = Conn.cursor()

To prove we are where we want to be, I'll inspect the tables we can see:

In [7]:
Cursor.execute('SHOW TABLES')
Cursor.fetchall()

[('company_names',),
 ('post_pan',),
 ('pre_pan',),
 ('running_data',),
 ('stock_prices',)]

To emphasise that we can more or less forget about python save a few objects and methods, and just focus on the SQL syntax, let's remind ourselves of what the 'stock_prices' table is structured like:

In [8]:
Cursor.execute('DESCRIBE stock_prices')
Cursor.fetchall()

[('Ticker', b'varchar(50)', 'YES', '', None, ''),
 ('Date_Added', b'date', 'YES', '', None, ''),
 ('Close', b'float', 'YES', '', None, '')]

Since pandas is the python analogue of SQL it's handy to compare the two and likely you'll also using pandas when using the python connector to manipulate your data in python. As a bit of a testament to the equivalence of pandas and SQL, the data returned by using `Cursor.fetchall()`, is perfectly structured to input directly into a DataFrame:

In [9]:
import pandas as pd

Cursor.execute('SELECT * FROM stock_prices')
SQL_Data = Cursor.fetchall()

SQL_to_Pandas = pd.DataFrame(SQL_Data, columns=['Ticker', 'Date', 'Stock Price'])
SQL_to_Pandas.head()

,Ticker,Date,Stock Price
0,MMM,2020-01-02,180.00
1,MMM,2020-01-03,178.45
2,MMM,2020-01-06,178.62
3,MMM,2020-01-07,177.90
4,MMM,2020-01-08,180.63


In [10]:
Query = 'SELECT company_name, date_added, Close FROM company_names INNER JOIN stock_prices ON company_names.ticker=stock_prices.ticker'
Cursor.execute(Query)
SQL_Data = Cursor.fetchall()

Joined_Data = pd.DataFrame(SQL_Data, columns=['Company_Name', 'Date', 'Stock Price'])
Joined_Data.head()

,Company_Name,Date,Stock Price
0,"""3M CO""",2020-01-02,180.00
1,"""3M CO""",2020-01-03,178.45
2,"""3M CO""",2020-01-06,178.62
3,"""3M CO""",2020-01-07,177.90
4,"""3M CO""",2020-01-08,180.63


## Aside on classes and objects in python

I have found that you often end up repeating statements and connections and cursors when using the python connector, enough to find creating a class that does some of the work for you useful. Obviously this will vary from project to project in terms of how complex the classes you create will be, in our case just a simple one will do.

In [11]:
class sql_api:
    
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.database = database
        self.Conn = Connector.connect(host=host, user=user, password=password, database=database)
        self.Cursor = self.Conn.cursor() 
        
    def describe(self, table):
        self.Cursor.execute('DESCRIBE {}'.format(table))
        return self.Cursor.fetchall()
    
    def show(self, arg):
        self.Cursor.execute('SHOW {}'.format(arg))
        return self.Cursor.fetchall()

- The general syntax for creating a class involves declaring the `class` followed by the class name. 
- __init__(self, args...) is a special method which is called when creating an instance of a class. It is used for giving specifiying objects.
- The argument 'self' is passed so that the object can bind attributes to itself; so it knows what data is attributed to it and not other objects. It is passed automatically when you call a method and there is no need to declare it when calling a method. It does the same job as '@' syntax in other languages. All attributes are declared in this sense: self.attribute = attribute, which demonstrates more explicitly what self means - it stores data to that object instance specifically.

To demonstrate the usefulness in this scenario, I will create an 'sql_api' object all the while setting up a connection to our database, and the run some simple queries without using `Cursor.execute` and `Cursor.fetchall()` multiple times. This is only slightly useful in our case but it's not hard to see how this can be useful in the longterm.

In [12]:
Cursor.close()
Conn.close()

In [13]:
kiara_db = sql_api('localhost', 'kiara', 'kiara_password', 'kiara_db')

In [14]:
kiara_db.show('tables')

[('company_names',),
 ('post_pan',),
 ('pre_pan',),
 ('running_data',),
 ('stock_prices',)]

In [15]:
kiara_db.describe('company_names')

[('Ticker', b'varchar(50)', 'YES', '', None, ''),
 ('SimFinId', b'int', 'YES', '', None, ''),
 ('Company_Name', b'varchar(150)', 'YES', '', None, ''),
 ('Industry_Id', b'int', 'YES', '', None, '')]

## Inserting data using python connector

We can now begin to see the use of the having this python driver by imagining we have collected data online through some python code. If we have an existing database we may want to automatically load the data into our database and so we need to use the python connector. I think it is also useful for manipulating and filtering data before storing it in our database. In this case I am using stored data, but you could imagine it has been freshly collected.

In [16]:
Running_Data = pd.read_csv('../Python/Exercise_Data/Running.csv')
Running_Data.head()

,Activity Type,Date,Favorite,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,Avg Run Cadence,...,Min Temp,Surface Interval,Decompression,Best Lap Time,Number of Laps,Max Temp,Moving Time,Elapsed Time,Min Elevation,Max Elevation
0,Running,2021-08-15 18:14:06,False,8.93,673,00:45:01,159,181,4.0,149,...,0.0,0:00,No,00:00.95,13,0.0,00:00:00,00:45:43,43,55
1,Running,2021-08-13 17:55:40,False,7.48,566,00:41:01,148,166,3.2,147,...,0.0,0:00,No,00:01.17,9,0.0,00:00:00,00:41:17,19,52
2,Running,2021-08-11 16:45:15,False,6.71,499,00:37:01,145,162,3.0,148,...,0.0,0:00,No,00:01.23,8,0.0,00:00:00,00:37:07,23,57
3,Running,2021-08-09 18:29:54,False,3.69,266,00:24:01,126,135,2.0,148,...,0.0,0:00,No,00:01.12,5,0.0,00:23:46,00:24:58,46,57
4,Running,2021-08-08 15:41:30,False,9.57,738,00:47:00,167,183,4.4,150,...,0.0,0:00,No,00:00.38,11,0.0,00:46:55,00:47:16,46,66


This is another use of using python first, is that we can edit the data before loading it, where as in SQL you'll need to load it and then edit it.

In [17]:
Running_Data.drop(columns=['Activity Type', 'Favorite', 'Min Temp', 'Max Temp', 'Surface Interval', 'Avg Stride Length',
                         'Decompression', 'Grit', 'Flow', 'Dive Time', 'Training Stress Score®', 'Aerobic TE', 'Best Pace',
                        'Avg Vertical Ratio', 'Avg Vertical Oscillation', 'Min Elevation', 'Max Elevation', 'Number of Laps',
                          'Avg Run Cadence', 'Max Run Cadence', 'Best Lap Time'], 
                inplace = True)
Running_Data.head()

,Date,Distance,Calories,Time,Avg HR,Max HR,Avg Pace,Elev Gain,Elev Loss,Moving Time,Elapsed Time
0,2021-08-15 18:14:06,8.93,673,00:45:01,159,181,5:02,11,13,00:00:00,00:45:43
1,2021-08-13 17:55:40,7.48,566,00:41:01,148,166,5:29,28,30,00:00:00,00:41:17
2,2021-08-11 16:45:15,6.71,499,00:37:01,145,162,5:31,33,35,00:00:00,00:37:07
3,2021-08-09 18:29:54,3.69,266,00:24:01,126,135,6:30,10,9,00:23:46,00:24:58
4,2021-08-08 15:41:30,9.57,738,00:47:00,167,183,4:55,28,31,00:46:55,00:47:16


We may also want to create a new database using python. To do so we need to see what datatypes we have so that these can correctly be loaded into our database.

In [18]:
Running_Data.dtypes

Date             object
Distance        float64
Calories         object
Time             object
Avg HR           object
Max HR           object
Avg Pace         object
Elev Gain        object
Elev Loss        object
Moving Time      object
Elapsed Time     object
dtype: object

We want to change some objects to integers and floats. The date and time values can stay as strings as SQL has functionality to turn these into datetime objects automatically, so long as we declare them to be those variables.

In [19]:
Running_Data = Running_Data.astype({'Distance':'float64', 'Calories':'int64', 'Avg HR':'int64', 'Max HR':'int64', 
                                    'Elev Gain':'int64', 'Elev Loss':'int64'})

ValueError: invalid literal for int() with base 10: '1,042'

Okay, I need to filter the data a little bit:

In [20]:
ToAmend = Running_Data[Running_Data['Calories'].str.contains(',')].copy() #Look for our value which contains a ','.
Index = ToAmend['Calories'].index[0] #Find the index
NewValue = ''.join(ToAmend['Calories'].iloc[0].split(',')) #Amend that value
Running_Data.iloc[Index, 2] = NewValue #Overwrite the old with the new

I've included this to demonstrate how different issues arise with data and how versitile pandas is to handle it. Also as a pandas refresher. Luckily, there was only one erroneous value, but in the case that this applied to many cells the above could be turned into a for loop or even a function for greater generality.

In [21]:
Running_Data.iloc[Index, 2] 

'1042'

We can try again:

In [22]:
Running_Data = Running_Data.astype({'Distance':'float64', 'Calories':'int64', 'Avg HR':'int64', 'Max HR':'int64', 
                                    'Elev Gain':'int64', 'Elev Loss':'int64'})

ValueError: invalid literal for int() with base 10: '--'

Similar thing as before, but with an easier mask:

In [23]:
Running_Data = Running_Data[Running_Data!='--']

In practice I would just rerun cell 21 again but to show all the different errors I have repeated the line.

In [24]:
Running_Data = Running_Data.astype({'Distance':'float64', 'Calories':'int64', 'Avg HR':'int64', 'Max HR':'int64', 
                                    'Elev Gain':'int64', 'Elev Loss':'int64'})

ValueError: cannot convert float NaN to integer

You can use `isna()` to return a boolean DataFrame for values which are NaN, and then follow it up with `.any()` to check if **any** columns contain a `True` (.all() returns something similar but on the condition that **all** values are `True`).

In [25]:
Running_Data.isna().any()

Date            False
Distance        False
Calories        False
Time            False
Avg HR           True
Max HR           True
Avg Pace        False
Elev Gain        True
Elev Loss        True
Moving Time     False
Elapsed Time    False
dtype: bool

In [26]:
Running_Data = Running_Data.dropna(axis=0)

I can rerun cell 25 to see if that's done the trick:

In [27]:
Running_Data.isna().any()

Date            False
Distance        False
Calories        False
Time            False
Avg HR          False
Max HR          False
Avg Pace        False
Elev Gain       False
Elev Loss       False
Moving Time     False
Elapsed Time    False
dtype: bool

Great, I should now be able to convert my datatypes:

In [28]:
Running_Data = Running_Data.astype({'Distance':'float64', 'Calories':'int64', 'Avg HR':'int64', 'Max HR':'int64', 
                                    'Elev Gain':'int64', 'Elev Loss':'int64'})

No error - good. Now I can go on and create and load the data into my database. For longer queries it is conventional, and easier, to separte your query as a string instead of just putting into `Cursor.execute()`. **Note:** using three commas creates a long string which can spread over lines.

In [29]:
Query = """CREATE Table Running_Data (
                Date DATETIME,
                Distance FLOAT,
                Calories INT,
                Time TIME,
                Avg_HR INT,
                Max_HR INT,
                Avg_Pace TIME,
                Elev_Gain INT,
                Elev_Loss INT,
                Moving_Time TIME,
                Elapsed_Time TIME
            )
"""

Conn = Connector.connect(host='localhost', user='kiara', password='kiara_password', database='kiara_db')
Cursor = Conn.cursor()

Cursor.execute(Query)
Conn.commit()

ProgrammingError: 1050 (42S01): Table 'running_data' already exists

`Conn.commit()` commits the transation to the database, altering its state. Cursor.execute on its own does not do this. To check that we have created our new table:

<img src="./Screenshots/Python_Connector.png" width=1000 height=1000 />

We can now load our data into table using `INSERT INTO TABLE VALUE (val1, val2, ..., valn)`. **Note:** this only works if you are adding data into each column. For specific columns use `INSERT INTO TABLE (col1, col2, ..., coln) VALUE (val1, val2, ..., valn)`

In [30]:
for i in range(Running_Data.shape[0]):
    Query = """INSERT INTO running_data VALUES (
                    '{}', {}, {}, '{}', {}, {}, 
                    '{}', {}, {}, '{}', '{}'
    )
    """.format(Running_Data.iloc[i][0], Running_Data.iloc[i][1], Running_Data.iloc[i][2], Running_Data.iloc[i][3],
               Running_Data.iloc[i][4], Running_Data.iloc[i][5], Running_Data.iloc[i][6], Running_Data.iloc[i][7],
               Running_Data.iloc[i][8], Running_Data.iloc[i][9], Running_Data.iloc[i][10])
    Cursor.execute(Query)
    Conn.commit()

We can now compare the data between MySQL and pandas:

<img src="./Screenshots/Running_Data.png" width=1000 height=1000 />

In [31]:
Running_Data.head(10)

,Date,Distance,Calories,Time,Avg HR,Max HR,Avg Pace,Elev Gain,Elev Loss,Moving Time,Elapsed Time
0,2021-08-15 18:14:06,8.93,673,00:45:01,159,181,5:02,11,13,00:00:00,00:45:43
1,2021-08-13 17:55:40,7.48,566,00:41:01,148,166,5:29,28,30,00:00:00,00:41:17
2,2021-08-11 16:45:15,6.71,499,00:37:01,145,162,5:31,33,35,00:00:00,00:37:07
3,2021-08-09 18:29:54,3.69,266,00:24:01,126,135,6:30,10,9,00:23:46,00:24:58
4,2021-08-08 15:41:30,9.57,738,00:47:00,167,183,4:55,28,31,00:46:55,00:47:16
5,2021-08-07 15:12:24,7.26,539,00:40:01,145,163,5:31,32,35,00:40:00,00:40:08
6,2021-08-01 12:55:47,6.98,571,00:39:17,155,183,5:37,174,175,00:38:59,00:40:17
7,2021-07-29 18:52:50,6.82,523,00:37:01,152,168,5:26,29,30,00:36:59,00:37:23
8,2021-07-26 17:02:04,6.74,520,00:37:02,150,165,5:30,30,35,00:36:44,00:37:20
9,2021-07-24 11:40:53,6.93,513,00:38:01,147,164,5:29,36,35,00:38:00,00:38:01


## Practical example

This is a practical example from a program I wrote to scrape job boards of new jobs. I have provided a few core functions which show you how python interacts a database through MySQL. This is to demonstrate how you can use pythons ability to progam something general and link it to SQL to store it in a database.

In [32]:
def FindNav(URL, Database, Username, Password, Load = False):

    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get(URL)

    NewPage = JobPages.Paginator(browser)
    Characteristic = NewPage.Navigator
    ID = NewPage.id

    browser.quit()

    if Load:
        conn, Cursor = Connect2db(Database, Username, Password)
        Cursor.execute("INSERT INTO clients(Name, Characteristic, ID) " \
                       "VALUES('{}', '{}', {})".format(URL, Characteristic, ID))
        conn.commit()

        Cursor.close()
        conn.close()

    return Characteristic, ID

In [33]:
def FindListing(URL, Database, Username, Password, Exp_Number = 1, Load = False):

    browser = webdriver.Chrome(ChromeDriverManager().install())
    browser.get(URL)
    browser.maximize_window()

    PrevHeight = browser.execute_script("return document.body.scrollHeight")

    Listing = JobPages.Job_Listings(browser, Exp_Number)

    Details = Listing.DisplayType
    Mode = Details[0]
    Element = Details[1]
    Tag = Details[2]
    TagName = Details[3]
    AttName = Details[4]

    if type(Element) == type(np.array([])):
        Element = 0

    browser.quit()

    if Load:

        conn, Cursor = Connect2db(Database, Username, Password)

        Cursor.execute("INSERT INTO jobhtmls(Name, Mode, Element, Tag, TagName, AttributeName) " \
                       "VALUES('{}', {}, {}, '{}', '{}', '{}')".format(URL, Mode, Element, Tag, TagName, AttName))

        conn.commit()

        Cursor.close()
        conn.close()

    return Details

In [34]:
def SearchJob(URL, BrowserURL, JobTypes, Database, Username, Password, PATH, JobIndex, MaxPage, KeywordSearch, Sort, FilteredSearch, SaveAs):

    try:
        WebPage = Utils.LoadURL()
        WebPage.Load(URL)
        browser = WebPage.browser
        FailSafe = Thread(target = Utils.CatchHang, args=(WebPage, ))
        FailSafe.start()

        browser.maximize_window()
    
        Utils.WaitForAccess(browser)

        #---> Establish connection to database of relevant HTML information.
        conn, Cursor = Connect2db(Database, Username, Password)

        #---> Pull over relevant HTML information.
        Cursor.execute("SELECT * FROM jobhtmls WHERE Name LIKE '%{}%'".format(URL.split('https://www.')[1].split('.co')[0]))
        Cache = Cursor.fetchall()[0]

        #---> Pull over navigator details.
        Cursor.execute("SELECT Characteristic, ID FROM clients WHERE Name LIKE '%{}%'".format(URL.split('https://www.')[1].split('.co')[0]))
        Characteristic, ID = Cursor.fetchall()[0]

        Cursor.close()
        conn.close()
        
    #---> Initialise to no jobs yet found.
    TotalNumberOfJobs = 0

    for Job in JobTypes:
        BrowserErrors = True
        ErrorCount = 0
        while BrowserErrors == True:
            if ErrorCount > 10:
                print('Error count exceeded 10, moving to next job keyword.')
                break

            PageNumber = 1
            Job = Job.title()
            try:
                CurrentURL = browser.current_url
                if KeywordSearch == True:
                    SearchBar.SearchFor(Job)
                if (Sort == True ) and (SortPossible == True):
                    SortPage.Sort()
                    ErrorID = '1 '

                ProceedToNextPage = True
                TotalNewJobs = []

                if BrowserURL != URL:
                    URL = BrowserURL
                    browser.get(BrowserURL)
        
                while (ProceedToNextPage == True) and (PageNumber <= MaxPage):
                    ErrorID = '2 '
                    NewJobs, ProceedToNextPage, FirstJobElement = FilterNewJobs(Cache, SaveAs, browser, Database, Username, Password, PATH, FilteredSearch=FilteredSearch)
                    ErrorID = '3 '
                    TotalNewJobs += NewJobs
                    print(len(TotalNewJobs))
                    if (ProceedToNextPage == True) and (PageNumber < MaxPage):    
                        ProceedToNextPage = JobPages.Clicker(browser, ID, Characteristic, Cache, FirstJobElement)
                        Utils.WaitForAccess(browser)
                    PageNumber += 1

            except exceptions.TimeoutException:
                ErrorCount += 1
                print('Timeout caught, rebooting webpage...')
                browser.quit()
                return True, CurrentURL, JobIndex
            except:
                ErrorCount += 1
                #---> This error occurs if the failsafe detects a pagehang. The page has been shutdown by the failsafe.
                if 'MaxRetryError' in str(sys.exc_info()[0]):
                        print('Pagehang caught, rebooting...')
                        return True, CurrentURL, JobIndex
                else:
                    Utils.SaveError(SaveAs, 'ID: '+ErrorID+SaveAs+' , {}'.format(URL), 'JobPullErrorAt', PATH)
                continue

            #---> This is only reached if neither page hangs nor errors are found.
            TotalNumberOfJobs += len(TotalNewJobs)
            JobIndex += 1
            BrowserErrors = False 

    browser.quit()

    return False, TotalNumberOfJobs, JobIndex

SyntaxError: invalid syntax (<ipython-input-34-bd30879cd185>, line 29)

In [35]:
def Add2db(URL, Job, JobTitle, Database, Username, Password, PATH):

    DateToday = date.today().strftime("%d/%m/%Y")

    conn, Cursor = Connect2db(Database, username = Username, password = Password)
    Link = Job.find_element_by_tag_name('a').get_attribute('href')

    try: 
        Cursor.execute("SELECT Job_Title, Location, Company FROM newjobs")
        Data = Cursor.fetchall()

        #For when the database is empty.
        if len(Data) == 0:
            Duplicate = False
        else:
            Duplicate, Proximity = Utils.DuplicateTest((Title, Location, Company), Data)

    except:
        Utils.SaveError('adding to database', 'Company: {} \n Job_Title: {} \n Link: {}'.format(Company, Title, Link), 'DBErrorAt', PATH)
        return None
    
    #---> Occurs if the job isn't in the database, i.e. a new job.
    if Duplicate == False: 
        Cursor.execute("INSERT INTO NewJobs(Job_Title, Location, Company, Salary, Link, Date_Added, Job_Type, Description) "\
                        'VALUES("{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}")'.format(Title, Location, Company, Pay, Link, DateToday, JobTitle.casefold(), Description))    
        conn.commit()

        Cursor.close()
        conn.close()

        return None

    elif Duplicate == True:

SyntaxError: unexpected EOF while parsing (<ipython-input-35-90d183d4fe0d>, line 33)